 Import Library

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

 Get Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 Patch Dataset

In [69]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/'
IMAGE_SIZE = [224, 224]
train='/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/'
val='/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/'

Imagenet weights

In [17]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [18]:
for layer in vgg16.layers:
    layer.trainable = False

Getting number of output classes

In [19]:
folders = glob('/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/*/')
folders

['/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/R/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/X/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/Z/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/V/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/U/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/Q/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/Y/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/T/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/S/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/W/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/I/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/L/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/J/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/O/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/H/',
 '/content/drive/MyDrive/Colab Notebooks/Citra BISINDO/P/',
 '/content/drive/MyDrive/Colab Notebooks

 Create a model

In [62]:
x = Flatten()(vgg16.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg16.input, outputs=prediction)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

Model Compile

In [24]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Set up Generator

In [68]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(val,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 312 images belonging to 26 classes.
Found 312 images belonging to 26 classes.


Model Fit

In [31]:
model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
10/10 [==============================] - 349s 36s/step - loss: 3.4769 - accuracy: 0.2212 - val_loss: 1.6131 - val_accuracy: 0.6090
Epoch 2/5
10/10 [==============================] - 373s 39s/step - loss: 1.1041 - accuracy: 0.7276 - val_loss: 0.3597 - val_accuracy: 0.9359
Epoch 3/5
10/10 [==============================] - 345s 36s/step - loss: 0.3740 - accuracy: 0.9199 - val_loss: 0.1364 - val_accuracy: 0.9744
Epoch 4/5
10/10 [==============================] - 375s 39s/step - loss: 0.1756 - accuracy: 0.9551 - val_loss: 0.0723 - val_accuracy: 0.9904
Epoch 5/5
10/10 [==============================] - 342s 36s/step - loss: 0.0836 - accuracy: 0.9808 - val_loss: 0.0374 - val_accuracy: 1.0000


In [38]:
import matplotlib as plt

Prediction

In [60]:
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()
class_names = tuple(train_ds.class_names)
for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(224, 224))
  x = tf.keras.utils.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  predictions = model.predict(images, batch_size=32)
  score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} \nWith a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

Saving wall white (3).jpg to wall white (3).jpg
This image most likely belongs to G 
With a 9.81 percent confidence.
